In [1]:
import tkinter as tk
from tkinter import ttk, messagebox
import sqlite3

# ---------- BASE DE DATOS ----------
def conectar_db():
    conn = sqlite3.connect("estudiantes.db")
    cursor = conn.cursor()
    cursor.execute("""
        CREATE TABLE IF NOT EXISTS estudiantes (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            nombre TEXT NOT NULL,
            edad INTEGER,
            correo TEXT
        )
    """)
    conn.commit()
    conn.close()

# ---------- FUNCIONES PRINCIPALES ----------
def agregar_estudiante():
    nombre = entry_nombre.get()
    edad = entry_edad.get()
    correo = entry_correo.get()

    if not nombre or not edad or not correo:
        messagebox.showwarning("Advertencia", "Todos los campos son obligatorios.")
        return

    try:
        edad = int(edad)
    except ValueError:
        messagebox.showerror("Error", "La edad debe ser un número.")
        return

    conn = sqlite3.connect("estudiantes.db")
    cursor = conn.cursor()
    cursor.execute("INSERT INTO estudiantes (nombre, edad, correo) VALUES (?, ?, ?)",
                   (nombre, edad, correo))
    conn.commit()
    conn.close()

    limpiar_campos()
    mostrar_estudiantes()
    messagebox.showinfo("Éxito", "Estudiante agregado correctamente.")

def mostrar_estudiantes():
    for row in tree.get_children():
        tree.delete(row)
    conn = sqlite3.connect("estudiantes.db")
    cursor = conn.cursor()
    cursor.execute("SELECT * FROM estudiantes")
    for fila in cursor.fetchall():
        tree.insert("", tk.END, values=fila)
    conn.close()

def eliminar_estudiante():
    seleccionado = tree.selection()
    if not seleccionado:
        messagebox.showwarning("Advertencia", "Seleccione un estudiante para eliminar.")
        return
    id_estudiante = tree.item(seleccionado)["values"][0]

    conn = sqlite3.connect("estudiantes.db")
    cursor = conn.cursor()
    cursor.execute("DELETE FROM estudiantes WHERE id=?", (id_estudiante,))
    conn.commit()
    conn.close()

    mostrar_estudiantes()
    messagebox.showinfo("Eliminado", "Estudiante eliminado correctamente.")

def limpiar_campos():
    entry_nombre.delete(0, tk.END)
    entry_edad.delete(0, tk.END)
    entry_correo.delete(0, tk.END)

# ---------- INTERFAZ GRÁFICA ----------
ventana = tk.Tk()
ventana.title("Registro de Estudiantes")
ventana.geometry("600x400")
ventana.configure(bg="#f0f0f0")

frame_form = ttk.LabelFrame(ventana, text="Datos del Estudiante")
frame_form.pack(fill="x", padx=10, pady=10)

ttk.Label(frame_form, text="Nombre:").grid(row=0, column=0, padx=5, pady=5)
entry_nombre = ttk.Entry(frame_form, width=30)
entry_nombre.grid(row=0, column=1, padx=5, pady=5)

ttk.Label(frame_form, text="Edad:").grid(row=1, column=0, padx=5, pady=5)
entry_edad = ttk.Entry(frame_form, width=30)
entry_edad.grid(row=1, column=1, padx=5, pady=5)

ttk.Label(frame_form, text="Correo:").grid(row=2, column=0, padx=5, pady=5)
entry_correo = ttk.Entry(frame_form, width=30)
entry_correo.grid(row=2, column=1, padx=5, pady=5)

frame_botones = ttk.Frame(ventana)
frame_botones.pack(fill="x", padx=10, pady=5)

ttk.Button(frame_botones, text="Agregar", command=agregar_estudiante).pack(side="left", padx=5)
ttk.Button(frame_botones, text="Eliminar", command=eliminar_estudiante).pack(side="left", padx=5)
ttk.Button(frame_botones, text="Actualizar", command=mostrar_estudiantes).pack(side="left", padx=5)
ttk.Button(frame_botones, text="Limpiar", command=limpiar_campos).pack(side="left", padx=5)

frame_tabla = ttk.Frame(ventana)
frame_tabla.pack(fill="both", expand=True, padx=10, pady=10)

tree = ttk.Treeview(frame_tabla, columns=("ID", "Nombre", "Edad", "Correo"), show="headings")
tree.heading("ID", text="ID")
tree.heading("Nombre", text="Nombre")
tree.heading("Edad", text="Edad")
tree.heading("Correo", text="Correo")
tree.pack(fill="both", expand=True)

# ---------- INICIO ----------
conectar_db()
mostrar_estudiantes()
ventana.mainloop()
